In [1]:
import os
import datetime
import time
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.loader.dataloader import DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from ClusterDataset import ClusterDataset
from training import *
from data_statistics import *

from IPython.display import display

from Transformer import Transformer
from lang import Lang

2025-05-20 10:25:30.641755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 10:25:30.662964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747729530.687052     553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747729530.694493     553 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 10:25:30.721340: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load the dataset
hist_folder = "/eos/user/c/czeh/histo/"
data_folder_training = "/eos/user/c/czeh/graph_data"
data_folder_test = "/eos/user/c/czeh/graph_data_test"

model_folder = "/eos/user/c/czeh/model"
dataset_training = ClusterDataset(data_folder_training, hist_folder)
dataset_test = ClusterDataset(data_folder_test, hist_folder, test=True)

In [4]:
print_dataset_statistics(dataset_training)

Number of events in training dataset: 299


/eos/home-i04/c/czeh/SWAN_projects/Graph_Linking/ClusterDataset.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(osp.join(self.processed_dir, f'data_{

Number of nodes: 8710
Mean Number of nodes: 29.130434782608695
Max Number of nodes: 66
Number of edges: 53836
Mean Number of edges: 180.05351170568562
Number of positive edges: 47377
Mean Number of positive edges: 158.45150756835938
Number of negative edges: 6459
Mean Number of negative edges: 21.602006912231445


In [5]:
print_dataset_statistics(dataset_test)

Number of events in training dataset: 100
Number of nodes: 2891
Mean Number of nodes: 28.91
Max Number of nodes: 62
Number of edges: 17805
Mean Number of edges: 178.05
Number of positive edges: 15627
Mean Number of positive edges: 156.27000427246094
Number of negative edges: 2178
Mean Number of negative edges: 21.780000686645508


In [6]:
train_dl = DataLoader(dataset_training, shuffle=True)
test_dl = DataLoader(dataset_test, shuffle=True)

In [7]:
input_length = 20
max_seq_length = 20
batch_size = 1
max_nodes = 66

In [8]:
# Generating data
converter = Lang(max_nodes)
vocab_size = converter.n_words

In [27]:
epochs = 1
d_model = 128
num_heads = 2
num_layers = 6
d_ff = 512
dropout = 0.1
feature_num = dataset_training.get(10).x.shape[1]-1

# Model, loss, and optimizer
# model = SimpleTransformer(embed_size, input_length, vocab_size).to(device)
model = Transformer(dataset_training.get(10).num_nodes+1, vocab_size, d_model, num_heads, num_layers, d_ff, feature_num, max_nodes, max_seq_length, dropout).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
# loss_obj = FocalLoss(alpha=0.45, gamma=2)


# Optionally introduce weight decay
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [34]:
losses_per_epoch = []
# Optionally introduce gradient clipping
# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

#Initialize Variables for EarlyStopping
best_loss = float('inf')
best_model_weights = None
patience = 10

# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    total_loss = 0
    for sample in tqdm(train_dl, desc="training"):
        length = converter.y2seq(np.array(sample[0].y_trans)).shape[0]-2
        
        if (length < input_length):
            length += 1
        
        sample_seq = np.array(sample[0].y_trans)
        
        input_tensor = torch.zeros((length, input_length), dtype=torch.long).to(device)
        target_tensor = torch.zeros((length, input_length), dtype=torch.long).to(device)
        edge_tensor = torch.zeros((length, sample.x.shape[0], feature_num), dtype=torch.float).to(device)
        for i in range(length):
            input_seq = converter.y2seq(sample_seq, seq_length=input_length+1, index=i-input_length+1)
            input_tensor[i, :] = torch.tensor(input_seq[:-1], dtype=torch.long)
            target_tensor[i, :] = torch.tensor(input_seq[1:], dtype=torch.long)
            edge_tensor[i, :] = torch.unsqueeze(sample.x[:, :-1], dim=0).float()
            
        optimizer.zero_grad()
        output = model(edge_tensor, input_tensor)
        loss = criterion(output.contiguous().view(-1, vocab_size), target_tensor.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    loss = total_loss / len(train_dl)
    print(f"Epoch {epoch+1}, Loss: {loss:.6f}")
    losses_per_epoch.append(loss)
        
    # Early stopping
#     if loss < best_loss:
#         best_loss = val_loss
#         best_model_weights = copy.deepcopy(model.state_dict())  # Deep copy here      
#         patience = 10  # Reset patience counter
#     else:
#         patience -= 1
#         if patience == 0:
#             break
            
# Load the best model weights
# model.load_state_dict(best_model_weights)

training:   2%|▏         | 6/299 [00:00<00:23, 12.23it/s]

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          5,  7],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,
          7,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,
          8,  9],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,  8,
          9, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,  8,  9,
         10, 11],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,  8,  9, 10,
         11, 12],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,  8,  9, 10, 11,
         12, 14],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  7,  8,  9, 10, 11, 12,
         14, 15],
        [ 

training:   6%|▌         | 18/299 [00:01<00:22, 12.54it/s]

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          5,  6],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,
          6,  7],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,
          7,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,  7,
          8,  9],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,  7,  8,
          9, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,  7,  8,  9,
         10, 11],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,  7,  8,  9, 10,
         11, 12],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  5,  6,  7,  8,  9, 10, 11,
         12, 13],
        [ 

training:   7%|▋         | 22/299 [00:01<00:23, 11.95it/s]

...

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1,  6],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          6,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,
          8,  9],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,
          9, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,  9,
         10, 11],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,  9, 10,
         11, 12],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,  9, 10, 11,
         12, 13],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,  9, 10, 11, 12,
         13, 15],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6,  8,  9, 10, 11, 12, 13,
         15, 16],
        [ 

training: 100%|█████████▉| 298/299 [00:24<00:00, 12.87it/s]

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1,  4],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          4,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,
          5,  3],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,
          3,  6],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  3,
          6,  7],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  3,  6,
          7,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  3,  6,  7,
          8,  9],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  3,  6,  7,  8,
          9, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  3,  6,  7,  8,  9,
         10, 11],
        [ 

training: 100%|██████████| 299/299 [00:24<00:00, 12.29it/s]

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          1,  4],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          4,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,
          5,  6],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,
          6,  3],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  6,
          3,  7],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  6,  3,
          7,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  6,  3,  7,
          8,  9],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  6,  3,  7,  8,
          9, 10],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  4,  5,  6,  3,  7,  8,  9,
         10, 11],
        [ 

In [11]:
correct = 0
model.eval() 
torch.no_grad()
i = 0
for sample in tqdm(test_dl, desc="test"):
    res = [converter.y2seq(sample_seq, seq_length=input_length, index=0)]
    input_seq = converter.y2seq(sample_seq, seq_length=input_length, index=0)
    input_tensor[i, :] = torch.tensor(input_seq, dtype=torch.long).to(device)
    
#     input_tensor = torch.zeros((len(sample_seq)+1, input_length), dtype=torch.long).to(device)
    target_tensor = torch.zeros((len(sample_seq)+1, input_length), dtype=torch.long).to(device)
    edge_tensor = torch.zeros((len(sample_seq)+1, sample.x.shape[0], feature_num), dtype=torch.float).to(device)
    
    predictions = model(edge_tensor, input_tensor)
    predicted_index = predictions.argmax(-1)
    predicted_number = predicted_index[0, -1].item()

    step = 0
    while (predicted_number != 2 and step < 100):  # Disable gradient computation for inference
        input_tensor = torch.roll(input_tensor, -1, dims=1)
        input_tensor[0, -1] = predicted_number
    
        predictions = model(edge_tensor, input_tensor)
        predicted_index = predictions.argmax(-1)  # Get the index of the max log-probability for the last position
        predicted_number = predicted_index[0, -1].item()  # Convert to Python number
        res.append(predicted_number)
        step += 1
     
#     input_tensor = torch.roll(input_tensor, -1, dims=1)
#     input_tensor[0, -1] = predicted_number 
#     check = converter.arr2seq(input_tensor[0]) == converter.arr2seq(converter.subseq2arr(sample[0].y, input_length, 0, input_length))
    
    print(f"Input Sequence: {sample[0].y_trans}")
    print(f"Predicted Sequence: {np.array(res)}")
    print(f"Predicted Sequence: {converter.seq2y(np.array(res))}")
#     print(f"Correct: {check}")
    
#     try:
#         origG = torch_geometric.utils.to_networkx(sample, to_undirected=True)
#         res_nodes, res_edges = sequenceToGraph(converter.arr2seq(input_tensor[0], ignoreTokens=True))
#         newData = Data(x=sample.x, num_nodes=sample.num_nodes, edge_index=torch.from_numpy(res_edges))
#         G = torch_geometric.utils.to_networkx(newData, to_undirected=True)

#         fig, ax = plt.subplots(1, 2)
#         nx.draw_circular(origG, with_labels=True, ax=ax[0])
#         nx.draw_circular(G, with_labels=True, ax=ax[1])
#     except:
#         print("incorrect sequence")
    
    
#     correct += check

# print(f"Percentage of correct guesses: {correct / (len(test_dl))}")

test:   0%|          | 0/100 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# torch.save({'epoch': epoch+1,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'training_loss': loss
#             }, "/eos/user/c/czeh/tranformer.pt")
